# Time Series Forecasting Model Tutorial

## Introduction

This tutorial shows **Model Developers** on how to use and configure the **Developer Framework** and the **MRM ValidMind Platform**. The following steps will guide you to **automatically** document and test Time Series Forecasting models: 

- Step 1: Connect Notebook to ValidMind Project 
- Step 2: Import Raw Data
- Step 3: Run Data Validation Test Suite on Raw Data
- Step 4: Preprocess Data
- Step 5: Run Data Validation Test Suite on Processed Data
- Step 6: Load Pre-Trained Models
- Step 7: Run Model Validation Test Suite on Models



## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
%pip install --upgrade validmind


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    api_secret="API_SECRET",
    project="xxxxxxxxxxxxxxxxxxxxxxxxx"
)

/Users/anilsorathiya/Library/Caches/pypoetry/virtualenvs/validmind-pPj8dHa5-py3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2023-07-22 12:27:23,768 - INFO(validmind.api_client): Connected to ValidMind. Project: Stock Price Prediction Model - Initial Validation (clhhzo21s006wl9rl0swhv40h)


## Step 1: Connect Notebook to ValidMind Project
Prepare the environment for our analysis by following the **Before you begin** section. First, **import** all necessary libraries and modules required for our analysis. Next, **connect** to the ValidMind MRM platform, which provides a comprehensive suite of tools and services for model validation.

Finally, define and **configure** the specific use case we are working on by setting up any required parameters, data sources, or other settings that will be used throughout the analysis.

#### Explore Test Suites, Test Plans and Tests

In [3]:
vm.test_suites.list_suites()

ID,Name,Description,Test Plans
binary_classifier_full_suite,BinaryClassifierFullSuite,Full test suite for binary classification models.,"tabular_dataset_description, tabular_data_quality, binary_classifier_metrics, binary_classifier_validation, binary_classifier_model_diagnosis"
binary_classifier_model_validation,BinaryClassifierModelValidation,Test suite for binary classification models.,"binary_classifier_metrics, binary_classifier_validation, binary_classifier_model_diagnosis"
tabular_dataset,TabularDataset,Test suite for tabular datasets.,"tabular_dataset_description, tabular_data_quality"
time_series_dataset,TimeSeriesDataset,Test suite for time series datasets.,"time_series_data_quality, time_series_univariate, time_series_multivariate"
time_series_model_validation,TimeSeriesModelValidation,Test suite for time series model validation.,"regression_model_description, regression_models_evaluation, time_series_forecast, time_series_sensitivity"


In [4]:
vm.test_plans.list_plans()

ID,Name,Description
binary_classifier_metrics,BinaryClassifierMetrics,Test plan for sklearn classifier metrics
binary_classifier_validation,BinaryClassifierPerformance,Test plan for sklearn classifier models
binary_classifier_model_diagnosis,BinaryClassifierDiagnosis,Test plan for sklearn classifier model diagnosis tests
tabular_dataset_description,TabularDatasetDescription,Test plan to extract metadata and descriptive statistics from a tabular dataset
tabular_data_quality,TabularDataQuality,Test plan for data quality on tabular datasets
time_series_data_quality,TimeSeriesDataQuality,Test plan for data quality on time series datasets
time_series_univariate,TimeSeriesUnivariate,Test plan to perform time series univariate analysis.
time_series_multivariate,TimeSeriesMultivariate,Test plan to perform time series multivariate analysis.
time_series_forecast,TimeSeriesForecast,Test plan to perform time series forecast tests.
time_series_sensitivity,TimeSeriesSensitivity,Test plan to perform time series forecast tests.


In [5]:
vm.tests.list_tests()

Test Type,Name,Description,ID
Metric,ModelMetadata,Custom class to collect the following metadata for a model: - Model architecture - Model hyperparameters - Model task type,validmind.model_validation.ModelMetadata
Metric,ClassifierOutOfSamplePerformance,Test that outputs the performance of the model on the test data.,validmind.model_validation.sklearn.ClassifierOutOfSamplePerformance
ThresholdTest,RobustnessDiagnosis,Test robustness of model by perturbing the features column values by adding noise within scale stardard deviation.,validmind.model_validation.sklearn.RobustnessDiagnosis
Metric,SHAPGlobalImportance,SHAP Global Importance,validmind.model_validation.sklearn.SHAPGlobalImportance
Metric,ConfusionMatrix,Confusion Matrix,validmind.model_validation.sklearn.ConfusionMatrix
Metric,ClassifierInSamplePerformance,Test that outputs the performance of the model on the training data.,validmind.model_validation.sklearn.ClassifierInSamplePerformance
ThresholdTest,OverfitDiagnosis,Test that identify overfit regions with high residuals by histogram slicing techniques.,validmind.model_validation.sklearn.OverfitDiagnosis
Metric,PermutationFeatureImportance,"The Feature Importance plot below calculates a score representing the importance of each feature in the model. A higher score indicates that the specific input feature will have a larger effect on the predictive power of the model. The importance score is calculated using Permutation Feature Importance. Permutation feature importance measures the decrease of model performance after the feature's values have been permuted, which breaks the relationship between the feature and the true outcome. A feature is ""important"" if shuffling its values increases the model error, because in this case the model relied on the feature for the prediction. A feature is ""unimportant"" if shuffling its values leaves the model error unchanged, because in this case the model ignored the feature for the prediction.",validmind.model_validation.sklearn.PermutationFeatureImportance
ThresholdTest,MinimumROCAUCScore,Test that the model's ROC AUC score on the validation dataset meets or exceeds a predefined threshold.,validmind.model_validation.sklearn.MinimumROCAUCScore
Metric,PrecisionRecallCurve,Precision Recall Curve,validmind.model_validation.sklearn.PrecisionRecallCurve


## Step 2: Import Raw Data

#### Import FRED Dataset

In [6]:
from validmind.datasets.regression import fred as demo_dataset

target_column = demo_dataset.target_column
feature_columns = demo_dataset.feature_columns

df = demo_dataset.load_data()
df.tail(10)

,MORTGAGE30US,FEDFUNDS,GS10,UNRATE
DATE,,,,
2023-03-02,6.65,NaN,NaN,NaN
2023-03-09,6.73,NaN,NaN,NaN
2023-03-16,6.60,NaN,NaN,NaN
2023-03-23,6.42,NaN,NaN,NaN
2023-03-30,6.32,NaN,NaN,NaN
2023-04-01,NaN,NaN,3.46,NaN
2023-04-06,6.28,NaN,NaN,NaN
2023-04-13,6.27,NaN,NaN,NaN
2023-04-20,6.39,NaN,NaN,NaN


## Step 3: Run Data Validation Test Suite on Raw Data

#### Explore the Time Series Dataset Test Suite

In [7]:
vm.test_suites.describe_suite("time_series_dataset")

ID,Name,Description,Test Plans
time_series_dataset,TimeSeriesDataset,Test suite for time series datasets.,"time_series_data_quality, time_series_univariate, time_series_multivariate"


#### Explore Test Plans

In [8]:
vm.test_plans.describe_plan("time_series_data_quality")

ID,Name,Description,Required Context,Tests
time_series_data_quality,TimeSeriesDataQuality,Test plan for data quality on time series datasets,['dataset'],TimeSeriesOutliers (ThresholdTest)TimeSeriesMissingValues (ThresholdTest)TimeSeriesFrequency (ThresholdTest)


In [9]:
vm.test_plans.describe_plan("time_series_univariate")

ID,Name,Description,Required Context,Tests
time_series_univariate,TimeSeriesUnivariate,Test plan to perform time series univariate analysis.,['dataset'],TimeSeriesLinePlot (Metric)TimeSeriesHistogram (Metric)ACFandPACFPlot (Metric)SeasonalDecompose (Metric)AutoSeasonality (Metric)AutoStationarity (Metric)RollingStatsPlot (Metric)AutoAR (Metric)AutoMA (Metric)


##### Connect Raw Dataset to ValidMind Platform

In [10]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
)

2023-07-22 12:27:24,345 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


##### Run Time Series Dataset Test Suite on Raw Dataset

In [11]:
config={
    
    # TIME SERIES DATA QUALITY PARAMS
    "time_series_outliers": {
        "zscore_threshold": 3,
    },
    "time_series_missing_values":{
        "min_threshold": 2,
    },
    
    # TIME SERIES UNIVARIATE PARAMS 
    "rolling_stats_plot": {
        "window_size": 12    
    },
     "seasonal_decompose": {
        "seasonal_model": 'additive'
    },
     "auto_seasonality": {
        "min_period": 1,
        "max_period": 3
    },
      "auto_stationarity": {
        "max_order": 3,
        "threshold": 0.05
    },
    "auto_ar": {
        "max_ar_order": 2
    },
    "auto_ma": {
        "max_ma_order": 2
    },

    # TIME SERIES MULTIVARIATE PARAMS 
    "lagged_correlation_heatmap": {
        "target_col": demo_dataset.target_column,
        "independent_vars": demo_dataset.feature_columns
    },
    "engle_granger_coint": {
        "threshold": 0.05
    },
}

full_suite = vm.run_test_suite(
    "time_series_dataset",
    dataset=vm_dataset,
    config = config,
)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


DatetimeIndex(['1947-01-01', '1947-02-01', '1947-03-01', '1947-04-01',
               '1947-05-01', '1947-06-01', '1947-07-01', '1947-08-01',
               '1947-09-01', '1947-10-01',
               ...
               '2023-03-02', '2023-03-09', '2023-03-16', '2023-03-23',
               '2023-03-30', '2023-04-01', '2023-04-06', '2023-04-13',
               '2023-04-20', '2023-04-27'],
              dtype='datetime64[ns]', name='DATE', length=3551, freq=None)


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
No frequency could be inferred for variable 'MORTGAGE30US'. Skipping seasonal decomposition and plots for this variable.
2023-07-22 12:27:33,935 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of FEDFUNDS: MS
2023-07-22 12:27:33,935 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of FEDFUNDS: MS
2023-07-22 12:27:34,253 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of GS10: MS
2023-07-22 12:27:34,253 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of GS10: MS
2023-07-22 12:27:34,527 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequenc

## Step 4: Preprocess Data

##### Handle Frequencies, Missing Values and Stationairty

In [12]:
# Sample frequencies to Monthly
resampled_df = df.resample("MS").last()

# Remove all missing values
nona_df = resampled_df.dropna()

# Take the first different across all variables
preprocessed_df = nona_df.diff().dropna()

## Step 5: Run Data Validation Test Suite on Processed Data

In [13]:
vm_dataset = vm.init_dataset(
    dataset=preprocessed_df,
    target_column=demo_dataset.target_column,
)

full_suite = vm.run_test_suite(
    "time_series_dataset",
    dataset=vm_dataset,
    config = config,
)

2023-07-22 12:28:15,866 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


DatetimeIndex(['1971-05-01', '1971-06-01', '1971-07-01', '1971-08-01',
               '1971-09-01', '1971-10-01', '1971-11-01', '1971-12-01',
               '1972-01-01', '1972-02-01',
               ...
               '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01',
               '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01',
               '2023-02-01', '2023-03-01'],
              dtype='datetime64[ns]', name='DATE', length=623, freq=None)


The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
2023-07-22 12:28:22,520 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of MORTGAGE30US: MS
2023-07-22 12:28:22,520 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of MORTGAGE30US: MS
2023-07-22 12:28:22,809 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of FEDFUNDS: MS
2023-07-22 12:28:22,809 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of FEDFUNDS: MS
2023-07-22 12:28:23,206 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of GS10: MS
2023-07-22 12:28:23,206 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of GS10: MS
2023-07-22 12:28:23,367 - INFO(validmind.tests.data_validation.SeasonalDecompose): Frequency of UNRATE: MS
2023-07-22 12:28:23,367 - INFO(valid

## Step 6: Load Pre-Trained Models

#### Load Pre-Trained Models

In [14]:
from validmind.datasets.regression import fred as demo_dataset
model_A, train_df_A, test_df_A = demo_dataset.load_model('fred_loan_rates_model_3')
model_B, train_df_B, test_df_B = demo_dataset.load_model('fred_loan_rates_model_4')

##### Connect a List of Models To the ValidMind Platform

In [15]:
# Initialize training and testing datasets for model A
vm_train_ds_A = vm.init_dataset(dataset=train_df_A, type="generic", target_column=demo_dataset.target_column)
vm_test_ds_A = vm.init_dataset(dataset=test_df_A, type="generic", target_column=demo_dataset.target_column)

# Initialize training and testing datasets for model B
vm_train_ds_B = vm.init_dataset(dataset=train_df_B, type="generic", target_column=demo_dataset.target_column)
vm_test_ds_B = vm.init_dataset(dataset=test_df_B, type="generic", target_column=demo_dataset.target_column)

# Initialize model A
vm_model_A = vm.init_model(
    model = model_A, 
    train_ds=vm_train_ds_A, 
    test_ds=vm_test_ds_A)

# Initialize model B
vm_model_B = vm.init_model(
    model = model_B,
    train_ds=vm_train_ds_B,
    test_ds=vm_test_ds_B)


list_of_models = [vm_model_A, vm_model_B]

2023-07-22 12:28:58,571 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-07-22 12:28:58,575 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-07-22 12:28:58,578 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-07-22 12:28:58,582 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


## Step 7: Run Model Validation Test Suite on Models

#### Explore the Time Series Model Validation Test Suite

In [16]:
vm.test_suites.describe_test_suite("time_series_model_validation")

ID,Name,Description,Test Plans
time_series_model_validation,TimeSeriesModelValidation,Test suite for time series model validation.,"regression_model_description, regression_models_evaluation, time_series_forecast, time_series_sensitivity"


#### Explore Test Plans

In [17]:
vm.test_plans.describe_plan("regression_model_description")

ID,Name,Description,Required Context,Tests
regression_model_description,RegressionModelDescription,Test plan for performance metric of regression model of statsmodels library,['model'],DatasetSplit (Metric)ModelMetadata (Metric)


In [18]:
vm.test_plans.describe_plan("regression_models_evaluation")

ID,Name,Description,Required Context,Tests
regression_models_evaluation,RegressionModelsEvaluation,Test plan for metrics comparison of regression model of statsmodels library,"['models', 'model']",RegressionModelsCoeffs (Metric)RegressionModelsPerformance (Metric)


In [19]:
vm.test_plans.describe_plan("time_series_forecast")

ID,Name,Description,Required Context,Tests
time_series_forecast,TimeSeriesForecast,Test plan to perform time series forecast tests.,['models'],RegressionModelForecastPlotLevels (Metric)


In [20]:
vm.test_plans.describe_plan("time_series_sensitivity")

ID,Name,Description,Required Context,Tests
time_series_sensitivity,TimeSeriesSensitivity,Test plan to perform time series forecast tests.,['models'],RegressionModelSensitivityPlot (Metric)


#### Run Model Validation Test Suite on a List of Models

In [21]:
config= {
    "regression_forecast_plot_levels": {
        "transformation": "integrate",
    },
    "regression_sensitivity_plot": {
        "transformation": "integrate",
        "shocks": [0.3],
    }
}

full_suite = vm.run_test_suite(
    "time_series_model_validation",
    model = vm_model_B,
    models = list_of_models,
    config = config,
)

2023-07-22 12:29:02,956 - INFO(validmind.tests.model_validation.statsmodels.RegressionModelSensitivityPlot): {'transformation': 'integrate', 'shocks': [0.3], 'regression_forecast_plot_levels': {'transformation': 'integrate'}}
2023-07-22 12:29:02,956 - INFO(validmind.tests.model_validation.statsmodels.RegressionModelSensitivityPlot): {'transformation': 'integrate', 'shocks': [0.3], 'regression_forecast_plot_levels': {'transformation': 'integrate'}}
